<table width="100%"><tr style="background-color:white;">
    <td style="text-align:left;padding:0px;width:142px'">
        <a href="https://qworld.net" target="_blank">
            <img src="../images/QWorld.png"></a></td>
    <td width="*">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
    <!-- ############################################# -->
    <td style="padding:0px;width:90px;">
        <img align="right" src="../images/follow_us.png" height="40px"></td>
    <td style="padding:0px;width:40px;">
        <a href="https://twitter.com/QWorld19" target="_blank">
        <img align="right" src="../images/Twitter.png" width="40px"></a> </td>
    <td style="padding:0px;width:5px;"></td>
    <td style="padding:0px;width:40px;">
        <a href="https://www.facebook.com/qworld19/" target="_blank">
        <img align="right" src="../images/Fb.png"></a></td>
    <td style="padding:0px;width:5px;"></td>
    <td style="padding:0px;width:40px;">
        <a href="https://www.linkedin.com/company/qworld19" target="_blank">
        <img align="right" src="../images/LinkedIn.png"></a></td>
    <td style="padding:0px;width:5px;"></td>
    <td style="padding:0px;width:40px;">
        <a href="https://www.youtube.com/qworld19" target="_blank">
        <img align="right" src="../images/YT.png"></a></td>
    <!-- ############################################# -->
    <td style="padding:0px;width:60px;">
        <img align="right" src="../images/join.png" height="40px"></td>
    <td style="padding:0px;width:40px;">
        <a href="https://discord.gg/akCvr7U87g"
           target="_blank">
        <img align="right" src="../images/Discord.png"></a></td>
    <!-- ############################################# -->
    <td style="padding:0px;width:72px;">
        <img align="right" src="../images/w3.png" height="40px"></td>
    <td style="padding:0px;width:40px;">
        <a href="https://qworld.net" target="_blank">
        <img align="right" src="../images/www.png"></a></td>
</tr></table>

<table width = "100%">
  <tr style="background-color:white;">
    <td style="text-align:right;vertical-align:bottom;font-size:12px;"> 
        Prepared by Anastasija Trizna (QPoland, QLatvia)</td>
    </tr>
    <tr><td align="right" style="color:#bbbbbb;background-color:#ffffff;font-size:11px;font-style:italic;">
        This cell contains some macros. If there is a problem with displaying mathematical formulas, please run this cell to load these macros.
    </td></tr>
 </table>
 
$ \newcommand{\bra}[1]{\langle #1|} $
$ \newcommand{\ket}[1]{|#1\rangle} $
$ \newcommand{\braket}[2]{\langle #1|#2\rangle} $
$ \newcommand{\dot}[2]{ #1 \cdot #2} $
$ \newcommand{\biginner}[2]{\left\langle #1,#2\right\rangle} $
$ \newcommand{\mymatrix}[2]{\left( \begin{array}{#1} #2\end{array} \right)} $
$ \newcommand{\myvector}[1]{\mymatrix{c}{#1}} $
$ \newcommand{\myrvector}[1]{\mymatrix{r}{#1}} $
$ \newcommand{\mypar}[1]{\left( #1 \right)} $
$ \newcommand{\mybigpar}[1]{ \Big( #1 \Big)} $
$ \newcommand{\sqrttwo}{\frac{1}{\sqrt{2}}} $
$ \newcommand{\dsqrttwo}{\dfrac{1}{\sqrt{2}}} $
$ \newcommand{\onehalf}{\frac{1}{2}} $
$ \newcommand{\donehalf}{\dfrac{1}{2}} $
$ \newcommand{\hadamard}{ \mymatrix{rr}{ \sqrttwo & \sqrttwo \\ \sqrttwo & -\sqrttwo }} $
$ \newcommand{\vzero}{\myvector{1\\0}} $
$ \newcommand{\vone}{\myvector{0\\1}} $
$ \newcommand{\vhadamardzero}{\myvector{ \sqrttwo \\  \sqrttwo } } $
$ \newcommand{\vhadamardone}{ \myrvector{ \sqrttwo \\ -\sqrttwo } } $
$ \newcommand{\myarray}[2]{ \begin{array}{#1}#2\end{array}} $
$ \newcommand{\X}{ \mymatrix{cc}{0 & 1 \\ 1 & 0}  } $
$ \newcommand{\Z}{ \mymatrix{rr}{1 & 0 \\ 0 & -1}  } $
$ \newcommand{\Htwo}{ \mymatrix{rrrr}{ \frac{1}{2} & \frac{1}{2} & \frac{1}{2} & \frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & \frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} & \frac{1}{2} } } $
$ \newcommand{\CNOT}{ \mymatrix{cccc}{1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0} } $
$ \newcommand{\norm}[1]{ \left\lVert #1 \right\rVert } $

 ---

## Complete BB84 Protocol Implementation

#### 0. Defining `NoisyChannel()` and other helpful functions

In [10]:
# import all necessary objects and methods for quantum circuits
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
import random
from random import randrange

#Initial source: awards/teach_me_qiskit_2018/cryptography/Cryptography.ipynb

#Code modified to introduce noise in communication channel

def NoisyChannel(qc1, qc2, qc1_name):
    ''' This function takes the output of a circuit qc1 (made up only of x and 
        h gates, simulate noisy quantum channel, where Pauli errors (X - bit flip; Z - phase flip
        will occur in qc2 and then initializes another circuit qc2 with introduce noise.
    ''' 
    
    # Quantum state is retrieved from qasm code of qc1
    qs = qc1.qasm().split(sep=';')[4:-1]

    # Process the code to get the instructions
    for index, instruction in enumerate(qs):
        qs[index] = instruction.lstrip()

    # Parse the instructions and apply to new circuit
    for instruction in qs:
        if instruction[0] == 'x':
            if instruction[5] == '[':
                old_qr = int(instruction[6:-1])
            else:
                old_qr = int(instruction[5:-1])
            qc2.x(qreg[old_qr])
        elif instruction[0] == 'h':
            if instruction[5] == '[':
                old_qr = int(instruction[6:-1])
            else:
                old_qr = int(instruction[5:-1])
            qc2.h(qreg[old_qr])
        elif instruction[0] == 'm': # exclude measuring:
            pass
        else:
            raise Exception('Unable to parse instruction')
    
    ### Introducing noise
    for instruction in qs:
        if randrange(7)<1:
            if instruction[5] == '[':
                old_qr = int(instruction[6:-1])
            else:
                old_qr = int(instruction[5:-1])
            qc2.x(qreg[old_qr]) #apply bit-flip error
        if randrange(7)<1:
            if instruction[5] == '[':
                old_qr = int(instruction[6:-1])
            else:
                old_qr = int(instruction[5:-1])
            qc2.z(qreg[old_qr]) #apply phase-flip error

In [11]:
def print_outcomes_in_reserve(counts): # takes a dictionary variable
    for outcome in counts: # for each key-value in dictionary
        reverse_outcome = ''
        for i in outcome: # each string can be considered as a list of characters
            reverse_outcome = i + reverse_outcome # each new symbol comes before the old symbol(s)
    return reverse_outcome

<h4>1. Distributing quantum states</h4>

Let's create a circuit with 24 qubits that Asja will send to Balvis.

In [12]:
qreg = QuantumRegister(24) # quantum register with 24 qubits
creg = ClassicalRegister(24) # classical register with 24 bits

send=[] #Initial bit string to send
asja_basis=[] #Register to save information about encoding basis
balvis_basis=[] #Register to save information about decoding basis

#Asja
asja = QuantumCircuit(qreg, creg, name='Asja')

for i in range(24):
    bit = randrange(2)
    send.append(bit)
for i, n in enumerate(send):
    if n==1: asja.x(qreg[i]) # apply x-gate
for i in range(24):
    r=randrange(2) #Asja randomly pick a basis
    if r==0: #if bit is 0, then she encodes in Z basis
        asja_basis.append('Z')
    else: #if bit is 1, then she encodes in X basis
        asja.h(qreg[i])
        asja_basis.append('X')

balvis = QuantumCircuit(qreg, creg, name='Balvis') #Defining Balvis circuit
NoisyChannel(asja, balvis, 'Asja') #Asja sends noisy states to Balvis

#Balvis 
for i in range(24):
    r=randrange(2) #Balvis randomly pick a basis
    if r==0: #if bit is 0, then measures in Z basis
        balvis.measure(qreg[i],creg[i])
        balvis_basis.append('Z')
    else: #if bit is 1, then measures in X basis
        balvis.h(qreg[i])
        balvis.measure(qreg[i],creg[i])
        balvis_basis.append('X')

job = execute(balvis,Aer.get_backend('qasm_simulator'),shots=1)
counts = job.result().get_counts(balvis)
counts = print_outcomes_in_reserve(counts)
received = list(map(int, counts))

<h4>2. Sifting</h4>

In [13]:
#Sifting
asja_key=[] #Asjas register for matching rounds
balvis_key=[] #Balvis register for matching rounds
for j in range(0,len(asja_basis)): #Going through list of bases 
    if asja_basis[j] == balvis_basis[j]: #Comparing
        asja_key.append(send[j])
        balvis_key.append(received[j]) #Keeping key bit if bases matched
    else:
        pass #Discard round if bases mismatched

<h4>3. QBER</h4>

In [14]:
#QBER
rounds = len(asja_key)//3
errors=0
for i in range(rounds):
    bit_index = randrange(len(asja_key)) 
    tested_bit = asja_key[bit_index]
    if asja_key[bit_index]!=balvis_key[bit_index]: #comparing tested rounds
        errors=errors+1 #calculating errors
    del asja_key[bit_index] #removing tested bits from key strings
    del balvis_key[bit_index]
QBER=errors/rounds #calculating QBER
QBER=round(QBER,2) #saving the answer to two decimal places

print("QBER value =", QBER)
print("Asjas secret key =", asja_key)
print("Balvis secret key =", balvis_key)

QBER value = 0.0
Asjas secret key = [1, 1, 0, 1, 0, 0, 1, 0, 0, 0]
Balvis secret key = [0, 1, 0, 1, 0, 0, 1, 1, 0, 1]


<h4>4. Information reconciliation</h4>

Let's suppose our **threshold value is 0.25**.

* If QBER $= 0$: Asja and Balvis discard Cascade protocol step.

* If $0 <$ QBER $< 0.25$: Perform Cascade protocol with error correction on remaining key bits.

* If QBER $\geq 0.25$: Asja and Balvis abort the protocol and restart.

**Let's create a program that will handle above described QBER value cases.** 

Recalling Cascade protocol,

<li>Asja and Balvis shuffle their bits (such that key bit indexes still match).
<li>They subdivide their key strings in blocks. Block size is $w_1=\frac{0.73}{QBER}$ (rounded to an integer). Last block can consist of remainder bits. 
<li>Both calculate parity bit for each block and compare. Blocks with matching parity are accepted as correct bits and saved in $K$<sub>final</sub> list.
<li>If parity do not match, they apply bisective search - second pass.

Asja and Balvis apply second pass of cascade protocol on blocks where parity bits do not match.

<li>They shuffle the bits.
<li>Divide blocks in half and calculate parity.
<li>Compare parity bits. Blocks with matching parity are accepted as correct bits and saved in $K$<sub>final</sub> list.
<li>If parity bits do not match, they apply bisective search and repeat second pass.

<b>Cascade protocol

In [15]:
def split(list1, n): 
    out = []
    last = 0.0
    while last < len(list1):
        out.append(list1[int(last):int(last + n)])
        last += n
    return out

def cascade_pass(lA, lB, n): #input key lists A-Asja, B-Balvis and target block size to divide in blocks
    #Shuffle
    permutation = list(zip(lA, lB)) #map the index of multiple lists
    random.shuffle(permutation) #performing permutation
    shuffledLA, shuffledLB = zip(*permutation) #unpacking values
    #Split
    splitLA=split(shuffledLA, n)
    splitLB=split(shuffledLB, n)
    #Calculate parity
    #Creating empty lists, where "correctA/B" will include blocks with no error found
    #And "errorA/B" list with blocks where parities mismatched
    correctA, correctB, errorA, errorB= [], [], [], []
    sumBlocksA = [sum(block) for block in splitLA] # calculating parity by first calculating sums of each block in splitA/B
    sumBlocksB = [sum(block) for block in splitLB]
    parityA = [i %2 for i in sumBlocksA] #then applying mod(2) operator to our calculated sums and saving results
    parityB = [i %2 for i in sumBlocksB] #in parity bit list
    for i,value in enumerate(range(len(parityA))): #comparing parity bits from list1 with list2
        if parityA[i]==parityB[i]: #if parity bits matched - we add corresponding blocks to our list 'correct'
            correctA.append(splitLA[i])
            correctB.append(splitLB[i])
        else:
            errorA.append(splitLA[i]) #if parity bits mismatched - we add corresponding blocks to our list 'errors'
            errorB.append(splitLB[i])
    keyA = [item for i in correctA for item in i] #Converting our correct blocks into a list
    keyB= [item for i in correctB for item in i]
    return keyA, keyB, errorA, errorB #returning key that consist of correct blocks (list) and blocks with errors (tuple)

In [16]:
#Before starting error correction, we check calculated QBER value
if QBER==0.0:
    print("QBER is 0. Cascade Protocol skipped!")
    print("Final Key Asja", asja_key)
    print("Final Key Balvis", balvis_key)
if QBER>=0.25: 
    print("QBER value is", QBER,"\nThreshold value reached! Protocol Aborted!") #If QBER is above threshold value - we abort protocol
if 0<QBER<=0.25: #if 0<QBER<=0.25 we perform Cascade protocol
    blockSize=0.73//QBER
    kFinalA, kFinalB=[], [] #creating registers for final keys
    #Cascade protocol 1st pass
    corrBlockA, corrBlockB, errBlockA, errBlockB=cascade_pass(asja_key, balvis_key, blockSize) #cascade function
    kFinalA.extend(corrBlockA) #adding block which parity bits matched to final key string
    kFinalB.extend(corrBlockB)
    
#We now aproximately know how many errors we have in initial key string,
#because after first pass each block in errorA/B list contains 1 (or other odd number) of errors
#We now can determine the final (corrected) key list length before we correct those errors (when 1 bit is left in each block)
#In other words, we will have a key length in penultimate pass of our Cascade protocol

    penultimatePassLength=len(asja_key)-len(errBlockA)
    while len(kFinalA)!=penultimatePassLength: #Bisective search at each block until corrected key length is not equal length of initial key minus error blocks number after first pass
        for i, (blockA, blockB) in enumerate(zip(errBlockA, errBlockB)):
            if len(blockA)>1:
                secondPassA=list(blockA)# we convert block into a lists
                secondPassB=list(blockB)
                blockSize2=len(blockA)//2 #we change block size, now we will divide each block that contains an error in halfs
                corrBlockA2, corrBlockB2,  errBlockA2, errBlockB2=cascade_pass(secondPassA, secondPassB, blockSize2) #and apply cascade
                kFinalA.extend(corrBlockA2) #then we will add correct bits to key strings
                kFinalB.extend(corrBlockB2)
                errBlockA[i]=errBlockA2[0] #updating error block values
                errBlockB[i]=errBlockB2[0]
            if len(blockA)==1: #Side case if one block in the round will be shorter than the oner thus will require less passes
                for bit in blockA:
                    if bit==1:
                        bitA=errBlockA[0][0]
                        kFinalA.append(bitA)#Asja adds corresponding bit to her key string without change
                        bitB=errBlockB[0][0]+1 #but Balvis will first correct the error by flipping the bit value 
                        kFinalB.append(bitB)
                    if bit==0:
                        bitA=errBlockA[0][0]
                        kFinalA.append(bitA) #Asja adds corresponding bit to her key string without change
                        bitB=errBlockB[0][0]-1 #but Balvis will first correct the error by flipping the bit value 
                        kFinalB.append(bitB)
                        
        #print("---PERFORMING NEXT PASS---\n", "Final key Asja:", kFinalA, "\n", "Final key Balvis", kFinalB)
        #print(" Blocks with errors Asja", errBlockA, "\n", "Blocks with errors Balvis", errBlockB)
        
    #After previous passes we have a nested lists, to convert them:    
    errorA=[item for elem in errBlockA for item in elem]
    errorB=[item for elem in errBlockB for item in elem]
    
    #Error correction step, when our error blocks contains just 1 bit (error)
    for i, error in enumerate(zip(errorA, errorB)):
#        bitA=int(''.join(map(str, errorA))) #Converting tuple to integer
#        bitB=int(''.join(map(str, errorB)))
        bitA=int(errorA[i])
        bitB=int(errorB[i])
        if bitA==1:
            kFinalA.append(bitA)
            correctedBitB=bitB+1
            kFinalB.append(correctedBitB)
        if bitA==0:
            kFinalA.append(bitA)
            correctedBitB=bitB-1
            kFinalB.append(correctedBitB)
            
    print("Final Key Asja", kFinalA)
    print("Final Key Balvis", kFinalB)

QBER is 0. Cascade Protocol skipped!
Final Key Asja [1, 1, 0, 1, 0, 0, 1, 0, 0, 0]
Final Key Balvis [0, 1, 0, 1, 0, 0, 1, 1, 0, 1]


But recall parity check problem! 

Even if we see QBER $=0$ or we implemented Cascade protocol and corrected errors, it doesn't mean that we can be 100% sure our key strings are identical. Some sneaky error bits can still be hiding there. 

Thus, it is important to run BICONF strategy for cases where QBER=0 and after implementing Cascade protocol. We need to be confident that our keys are identical!

<b>BICONF strategy

We will run BICONF strategy if:
* QBER $=0$ or
* QBER $< 0.25$ right after we finished implementing Cascade protocol and corrected the errors.

For case where QBER $=0$, we will use fixed block size value - 8 bits per block.

And we will run $8$ rounds before we conclude that our keys are correct. 

In [17]:
from numpy import log as ln

if QBER!=0: #defining size of blocks
    biconfBlockSize=(4*ln(2))//(3*QBER)
if QBER==0:
    biconfBlockSize=8
kFinalA=asja_key
kFinalB=balvis_key
#print(QBER)

rounds = 0 #counting rounds
biconfError=[] #creating register for rounds with an error
error=0 #register for found and corrected error

while rounds!=8: #we will go through rounds and monitor if blocks with errors will be found 
    rounds=rounds+1
    #Creating random subsets
    kFinalZipped=list(zip(kFinalA, kFinalB)) #maping indexes of our two lists
    randomBlock=random.sample(list(enumerate(kFinalZipped)), int(biconfBlockSize))
    #at this point we will have nested tuple that contains (index of random bit, (bit from Asja string, bit from Balvis string))
    #print(randomBitList) #will print out our nested tuple
    #print(randomBitList[0]) # will print out one block (index, (bitA, bitB))
    #print(randomBitList[0][0]) # will print only first pair index
    #print(randomBitList[0][1][0]) #will print only first pair Asjas' bit
    
    #We will now need to calculate and compare parity bits for both users bits
    sumBlockA=0
    sumBlockB=0
    for i in range(0,int(biconfBlockSize)):
        sumBlockA=sumBlockA+randomBlock[i][1][0]
        sumBlockB=sumBlockB+randomBlock[i][1][1]
    parityA = sumBlockA%2 #then aplying mod(2) operator to our calculated sums and saving results
    parityB = sumBlockB%2
    
    if parityA!=parityB: #if parities of block dismatch - we bisective search to correct error before continue with next round
        print("Error found in round:", rounds)
        print("Applying bisective search and error correction")
        #Applying bisective search to find and correct an error
        while len(randomBlock)>1: #We will take our block with error and run besective search till we find bit with error
            #Split the block
            if len(randomBlock)%2==1: #If block size is odd
                half=len(randomBlock)//2+1 #Lenght of our first block should be half+1
            else:
                half=len(randomBlock)//2
            splitBlock=split(randomBlock, half) #spliting our block in two parts
            for i, block in enumerate(splitBlock): #For each part
                sumA=0
                sumB=0
                for j in range(0,len(block)): #calculating sums 
                    sumA=sumA+splitBlock[i][j][1][0]
                    sumB=sumB+splitBlock[i][j][1][1]
                parA=sumA%2 #then calculate parities
                parB=sumB%2
                if parA==parB:
                    pass
                if parA!=parB: #if parities dismatch- we update our block and run while loop again
                    randomBlock=splitBlock[i]
        if len(randomBlock)==1: #once we isolate the error to 1 bit
            error=error+1
            print("Error found in bit:", randomBlock[0][0]) #we retrieving the index of bit pair
            errorIndex=int(randomBlock[0][0])
            #Apply error correction at Balvis' initial key string
        if kFinalB[errorIndex]==0:
            kFinalB[errorIndex]=1
        else:
            kFinalB[errorIndex]=0
        print("Error corrected!\n")
    else: #If parities matched
        pass

print("BICONF strategy completed!\n", error, "errors found!")
print("Final key Asja", kFinalA)
print("Final key Balvis", kFinalB)

Error found in round: 2
Applying bisective search and error correction
Error found in bit: 7
Error corrected!

Error found in round: 5
Applying bisective search and error correction
Error found in bit: 0
Error corrected!

Error found in round: 6
Applying bisective search and error correction
Error found in bit: 9
Error corrected!

BICONF strategy completed!
 3 errors found!
Final key Asja [1, 1, 0, 1, 0, 0, 1, 0, 0, 0]
Final key Balvis [1, 1, 0, 1, 0, 0, 1, 0, 0, 0]


<h4>5. Privacy Amplification</h4>

In [22]:
#Privacy amplification
import hashlib

#Generating seed (salt)
seed=[]
for i in kFinalA:
    a=randrange(2)
    seed.append(a)

#Adding seeds to the keys
kFinalA.append(seed)
kFinalB.append(seed)

#Converting lists to strings
strKFinalA = ' '.join([str(elem) for elem in kFinalA])
strKFinalB = ' '.join([str(elem) for elem in kFinalB])

#checking first bit to decide hash function to use
if kFinalA[0]==1:
    resultA=hashlib.sha256(strKFinalA.encode())
    print("Asjas' final key:", bin(int(resultA.hexdigest(), 16))[2:])
else:
    resultA=hashlib.sha3_256(strKFinalA.encode())
    print("Asjas' final key:", bin(int(resultA.hexdigest(), 16))[2:])

print()
if kFinalB[0]==1:
    resultB=hashlib.sha256(strKFinalB.encode())
    print("Balvis' final key:", bin(int(resultB.hexdigest(), 16))[2:])
else:
    resultB=hashlib.sha3_256(strKFinalB.encode())
    print("Balvis' final key:", bin(int(resultB.hexdigest(), 16))[2:])

Asjas' final key: 1100000011001011100101111111011000101011110100011011100011100010111100001001010010000000101011010000100111111000001100111000100010010010111010101110011100110000011010010011010100100010111001000000111000011110011000000110010101001010000101110110111110110001

Balvis' final key: 1100000011001011100101111111011000101011110100011011100011100010111100001001010010000000101011010000100111111000001100111000100010010010111010101110011100110000011010010011010100100010111001000000111000011110011000000110010101001010000101110110111110110001


Next: [Man-in-the-middle Attack](QC16_Man-in-the-Middle.ipynb)